# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_data_df = pd.read_csv('weather_data_cleaned.csv')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [82]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_data_df[["Lat", "Lng"]].astype(float)

humidity = weather_data_df["Humidity"].astype(float)

In [83]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [59]:
ideal_weather_data_df = weather_data_df[weather_data_df["Max Temp"] > 20]
ideal_weather_data_df = ideal_weather_data_df[ideal_weather_data_df["Max Temp"] < 30]

ideal_weather_data_df = ideal_weather_data_df[ideal_weather_data_df["Wind Speed"] < 7]

ideal_weather_data_df = ideal_weather_data_df[ideal_weather_data_df["Cloudiness"] == 0]

ideal_weather_data_df = ideal_weather_data_df[ideal_weather_data_df["Humidity"] < 50]

ideal_weather_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date/Time
14,abbeville,50.1000,1.8333,29.16,30,0,4.19,FR,1660332626
19,sayat,38.7839,63.8803,28.93,24,0,6.66,TM,1660332777
56,jamestown,42.0970,-79.2353,23.36,37,0,5.66,US,1660332788
67,broome,42.2506,-75.8330,25.26,39,0,6.17,US,1660332791
136,dasoguz,41.8363,59.9666,25.97,41,0,4.12,TM,1660332812
251,qarqin,37.4185,66.0436,28.66,20,0,2.97,AF,1660332846
267,alto araguaia,-17.3147,-53.2153,27.38,14,0,1.82,BR,1660332851
339,cordoba,-31.4135,-64.1811,25.47,23,0,6.17,AR,1660332806
345,saint-francois,46.4154,3.9054,27.69,26,0,2.57,FR,1660332825
360,dunmore,41.4198,-75.6324,27.28,30,0,3.60,US,1660332879


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [75]:
ideal_weather_data_df["Hotel Name"] = ""

# use iterrows to iterate through pandas dataframe
for index, row in ideal_weather_data_df.iterrows():
    
    try:
        # geocoordinates
        # target_coordinates = "45, -93"
        target_coordinates = ( str(ideal_weather_data_df.loc[index, "Lat"]) + " , " + str(ideal_weather_data_df.loc[index, "Lng"]) )
        target_search = "hotel"
        target_radius = 5000
        target_type = "hotels"

        # set up a parameters dictionary
        params = {
            "location": target_coordinates,
            "keyword": target_search,
            "radius": target_radius,
            "type": target_type,
            "key": g_key
        }

        # base url
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        # run a request using our params dictionary
        response = requests.get(base_url, params=params)

        # response["results"]["name"]

        places_data = response.json()

        # print(json.dumps(places_data, indent=4, sort_keys=True)) 

        ideal_weather_data_df.loc[index, "Hotel Name"] = places_data["results"][0]["name"]
    
    except:
        ideal_weather_data_df.loc[index, "Hotel Name"] = "No hotels found!"
ideal_weather_data_df = ideal_weather_data_df[ideal_weather_data_df["Hotel Name"] != "No hotels found!"]
ideal_weather_data_df

C:\Users\Nathan\AppData\Local\Temp\ipykernel_7016\1484693758.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather_data_df["Hotel Name"] = ""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date/Time,Hotel Name
14,abbeville,50.1000,1.8333,29.16,30,0,4.19,FR,1660332626,Hôtel Mercure Abbeville Centre - Porte de la B...
56,jamestown,42.0970,-79.2353,23.36,37,0,5.66,US,1660332788,Chautauqua Harbor Hotel
136,dasoguz,41.8363,59.9666,25.97,41,0,4.12,TM,1660332812,Uzboý Oteli
267,alto araguaia,-17.3147,-53.2153,27.38,14,0,1.82,BR,1660332851,Hotel Karajás
339,cordoba,-31.4135,-64.1811,25.47,23,0,6.17,AR,1660332806,Yrigoyen 111 Hotel
345,saint-francois,46.4154,3.9054,27.69,26,0,2.57,FR,1660332825,Maison Vin d'Hus
360,dunmore,41.4198,-75.6324,27.28,30,0,3.60,US,1660332879,Radisson Lackawanna Station Hotel Scranton
367,mayo,38.8876,-76.5119,29.64,48,0,2.57,US,1660332881,Pirates Cove Restaurant and Dock Bar
377,ladario,-19.0047,-57.6017,29.16,30,0,5.66,BR,1660332885,Nacional Palace Hotel
401,pallasovka,50.0500,46.8833,25.35,37,0,5.43,RU,1660332893,Hotel Yubileinaya


In [77]:
# Cut it down a little if the list is too long


if (len(ideal_weather_data_df) > 10):
    ideal_weather_data_df = ideal_weather_data_df[ideal_weather_data_df["Max Temp"] < 27]

if (len(ideal_weather_data_df) > 10):
    ideal_weather_data_df = ideal_weather_data_df[ideal_weather_data_df["Max Temp"] < 26]

if (len(ideal_weather_data_df) > 10):
    ideal_weather_data_df = ideal_weather_data_df[ideal_weather_data_df["Max Temp"] < 25]

if (len(ideal_weather_data_df) > 10):
    ideal_weather_data_df = ideal_weather_data_df[ideal_weather_data_df["Max Temp"] < 24]

if (len(ideal_weather_data_df) > 10):
    ideal_weather_data_df = ideal_weather_data_df[ideal_weather_data_df["Max Temp"] < 23]
    
if (len(ideal_weather_data_df) > 10):
    ideal_weather_data_df = ideal_weather_data_df[ideal_weather_data_df["Max Temp"] < 22]


hotel_df = ideal_weather_data_df

In [78]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [80]:

coords = weather_data_df[["Lat", "Lng"]].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(coords, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)



fig

Figure(layout=FigureLayout(height='420px'))